<a href="https://colab.research.google.com/github/LucasMirandaVS/projeto_SQL_analysis/blob/main/DB_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install -q imdb-sqlite

In [2]:
import pandas as pd
import sqlite3


# Importando os Dados e Conectando ao banco SQLite

In [3]:
# df_mortes = pd.read_excel('CovidDeaths.xlsx') 

In [4]:
df_mortes = pd.read_excel('owid-covid-data.xlsx') 

In [5]:
type(df_mortes)

pandas.core.frame.DataFrame

In [6]:
con = sqlite3.connect('covid.db')

In [7]:
df_mortes.to_sql(name = 'Mortes', con = con, if_exists = 'replace', index = True)
con.commit()

Agora vou responder algumas perguntas de negócio pra análise

#Análise

#### Separando as variáveis que serão analisadas

In [8]:
consulta1 = 'Select Location, date, total_cases, new_cases, total_deaths, population From Mortes Where continent is not null order by 1,2'
resultado1 = pd.read_sql_query(consulta1, con)

In [9]:
display(resultado1)

,location,date,total_cases,new_cases,total_deaths,population
0,Afghanistan,2020-02-24,5.0,5.0,NaN,40099462.0
1,Afghanistan,2020-02-25,5.0,0.0,NaN,40099462.0
2,Afghanistan,2020-02-26,5.0,0.0,NaN,40099462.0
3,Afghanistan,2020-02-27,5.0,0.0,NaN,40099462.0
4,Afghanistan,2020-02-28,5.0,0.0,NaN,40099462.0
...,...,...,...,...,...,...
195097,Zimbabwe,2022-08-03,256423.0,20.0,5579.0,15993524.0
195098,Zimbabwe,2022-08-04,256444.0,21.0,5581.0,15993524.0
195099,Zimbabwe,2022-08-05,256444.0,0.0,5581.0,15993524.0
195100,Zimbabwe,2022-08-06,256444.0,0.0,5581.0,15993524.0


#### Quais os países com o maior número de mortes?

In [12]:
consulta9 = '''
Select Location, MAX(cast(Total_deaths as int)) as TotalDeathCount
From Mortes
Where continent is not null 
Group by Location
order by TotalDeathCount desc
'''

In [13]:
resultado9 = pd.read_sql_query(consulta9, con)
display(resultado9)

,location,TotalDeathCount
0,United States,1034152.0
1,Brazil,679996.0
2,India,526689.0
3,Russia,374983.0
4,Mexico,328320.0
...,...,...
226,Turkmenistan,NaN
227,Tuvalu,NaN
228,United States Virgin Islands,NaN
229,Vatican,NaN


Estados Unidos, Brasil, India, Russia e México são os países com mais mortes por covid. Chama atenção o Brasil ter mais mortes que a india, mesmo tendo 1/5 da população indiana.

#### Qual a taxa de letalidade da covid no Brasil? e no Mundo?

In [14]:
consulta3 = '''
Select Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From Mortes
Where location like '%Brazil%'
and continent is not null 
order by 1,2
'''

In [15]:
resultado3 = pd.read_sql_query(consulta3, con)
display(resultado3)

,location,date,total_cases,total_deaths,DeathPercentage
0,Brazil,2020-02-26,1.0,NaN,NaN
1,Brazil,2020-02-27,1.0,NaN,NaN
2,Brazil,2020-02-28,1.0,NaN,NaN
3,Brazil,2020-02-29,2.0,NaN,NaN
4,Brazil,2020-03-01,2.0,NaN,NaN
...,...,...,...,...,...
889,Brazil,2022-08-03,33924061.0,679275.0,2.002340
890,Brazil,2022-08-04,33964494.0,679536.0,2.000725
891,Brazil,2022-08-05,33994470.0,679758.0,1.999613
892,Brazil,2022-08-06,34011173.0,679939.0,1.999164


No dia 07/08 (data final da amostra), a taxa de mortalidade da covid no brasil era de aproximadamente 1,99. Isso significa que quem era diagnosticado com covid nesse dia, tinha mais ou menos 2% de chance de vir a óbito.

In [16]:
# Vendo a taxa pra américa do sul
consulta3_1 = '''
Select Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From Mortes
Where location like '%South America%'
order by 1,2
'''

In [17]:
resultado3_1 = pd.read_sql_query(consulta3_1, con)
display(resultado3_1)

,location,date,total_cases,total_deaths,DeathPercentage
0,South America,2020-02-22,NaN,NaN,NaN
1,South America,2020-02-23,2.0,NaN,NaN
2,South America,2020-02-24,2.0,NaN,NaN
3,South America,2020-02-25,2.0,NaN,NaN
4,South America,2020-02-26,3.0,NaN,NaN
...,...,...,...,...,...
893,South America,2022-08-03,62335180.0,1316484.0,2.111944
894,South America,2022-08-04,62419358.0,1317102.0,2.110086
895,South America,2022-08-05,62480221.0,1317510.0,2.108683
896,South America,2022-08-06,62520869.0,1317769.0,2.107727


Taxa levemente superior à brasileira.

In [18]:
consulta4 = '''
Select Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From Mortes
Where location like '%World%'
order by 1,2
'''

In [19]:
resultado4 = pd.read_sql_query(consulta4, con)
display(resultado4)

,location,date,total_cases,total_deaths,DeathPercentage
0,World,2020-01-22,557.0,17.0,3.052065
1,World,2020-01-23,657.0,18.0,2.739726
2,World,2020-01-24,944.0,26.0,2.754237
3,World,2020-01-25,1437.0,42.0,2.922756
4,World,2020-01-26,2120.0,56.0,2.641509
...,...,...,...,...,...
924,World,2022-08-03,580597930.0,6409778.0,1.103996
925,World,2022-08-04,582015605.0,6412803.0,1.101827
926,World,2022-08-05,582928015.0,6415652.0,1.100591
927,World,2022-08-06,584102431.0,6417879.0,1.098759


In [20]:
# India
consulta10 = '''
Select Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From Mortes
Where location like '%India%'
and continent is not null 
order by 1,2
'''

In [21]:
resultado10 = pd.read_sql_query(consulta10, con)
display(resultado10)

,location,date,total_cases,total_deaths,DeathPercentage
0,India,2020-01-30,1.0,NaN,NaN
1,India,2020-01-31,1.0,NaN,NaN
2,India,2020-02-01,1.0,NaN,NaN
3,India,2020-02-02,2.0,NaN,NaN
4,India,2020-02-03,3.0,NaN,NaN
...,...,...,...,...,...
916,India,2022-08-03,44087037.0,526530.0,1.194297
917,India,2022-08-04,44107588.0,526600.0,1.193899
918,India,2022-08-05,44126994.0,526649.0,1.193485
919,India,2022-08-06,44145709.0,526689.0,1.193070


#### Qual o percentual de infectados no Brasil? e no Mundo?

In [22]:
consulta5 = '''
Select Location, date, Population, total_cases,  (total_cases/population)*100 as PercentPopulationInfected
From Mortes
Where location like '%Brazil%'
order by 1,2
'''

In [23]:
resultado5 = pd.read_sql_query(consulta5, con)
display(resultado5)

,location,date,population,total_cases,PercentPopulationInfected
0,Brazil,2020-02-26,214326223.0,1.0,4.665785e-07
1,Brazil,2020-02-27,214326223.0,1.0,4.665785e-07
2,Brazil,2020-02-28,214326223.0,1.0,4.665785e-07
3,Brazil,2020-02-29,214326223.0,2.0,9.331569e-07
4,Brazil,2020-03-01,214326223.0,2.0,9.331569e-07
...,...,...,...,...,...
889,Brazil,2022-08-03,214326223.0,33924061.0,1.582824e+01
890,Brazil,2022-08-04,214326223.0,33964494.0,1.584710e+01
891,Brazil,2022-08-05,214326223.0,33994470.0,1.586109e+01
892,Brazil,2022-08-06,214326223.0,34011173.0,1.586888e+01


In [24]:
# India
consulta5_1 = '''
Select Location, date, Population, total_cases,  (total_cases/population)*100 as PercentPopulationInfected
From Mortes
Where location like '%India%'
order by 1,2
'''

In [25]:
resultado5_1 = pd.read_sql_query(consulta5_1, con)
display(resultado5_1)

,location,date,population,total_cases,PercentPopulationInfected
0,India,2020-01-30,1.407564e+09,1.0,7.104473e-08
1,India,2020-01-31,1.407564e+09,1.0,7.104473e-08
2,India,2020-02-01,1.407564e+09,1.0,7.104473e-08
3,India,2020-02-02,1.407564e+09,2.0,1.420895e-07
4,India,2020-02-03,1.407564e+09,3.0,2.131342e-07
...,...,...,...,...,...
916,India,2022-08-03,1.407564e+09,44087037.0,3.132152e+00
917,India,2022-08-04,1.407564e+09,44107588.0,3.133612e+00
918,India,2022-08-05,1.407564e+09,44126994.0,3.134991e+00
919,India,2022-08-06,1.407564e+09,44145709.0,3.136320e+00


In [26]:
consulta6 = '''
Select Location, date, Population, total_cases,  (total_cases/population)*100 as PercentPopulationInfected
From Mortes
Where location like '%South America%'
order by 1,2
'''

In [27]:
resultado6 = pd.read_sql_query(consulta6, con)
display(resultado6)

,location,date,population,total_cases,PercentPopulationInfected
0,South America,2020-02-22,433956670.0,NaN,NaN
1,South America,2020-02-23,433956670.0,2.0,4.608755e-07
2,South America,2020-02-24,433956670.0,2.0,4.608755e-07
3,South America,2020-02-25,433956670.0,2.0,4.608755e-07
4,South America,2020-02-26,433956670.0,3.0,6.913133e-07
...,...,...,...,...,...
893,South America,2022-08-03,433956670.0,62335180.0,1.436438e+01
894,South America,2022-08-04,433956670.0,62419358.0,1.438378e+01
895,South America,2022-08-05,433956670.0,62480221.0,1.439780e+01
896,South America,2022-08-06,433956670.0,62520869.0,1.440717e+01


In [28]:
consulta7= '''
Select Location, date, Population, total_cases,  (total_cases/population)*100 as PercentPopulationInfected
From Mortes
Where location like '%World%'
order by 1,2
'''

In [29]:
resultado7 = pd.read_sql_query(consulta7, con)
display(resultado7)

,location,date,population,total_cases,PercentPopulationInfected
0,World,2020-01-22,7.909295e+09,557.0,0.000007
1,World,2020-01-23,7.909295e+09,657.0,0.000008
2,World,2020-01-24,7.909295e+09,944.0,0.000012
3,World,2020-01-25,7.909295e+09,1437.0,0.000018
4,World,2020-01-26,7.909295e+09,2120.0,0.000027
...,...,...,...,...,...
924,World,2022-08-03,7.909295e+09,580597930.0,7.340704
925,World,2022-08-04,7.909295e+09,582015605.0,7.358628
926,World,2022-08-05,7.909295e+09,582928015.0,7.370164
927,World,2022-08-06,7.909295e+09,584102431.0,7.385012


#### Quais os países com as maiores taxas de infecção?

In [30]:
consulta8 = '''
Select Location, Population, MAX(total_cases) as HighestInfectionCount,  Max((total_cases/population))*100 as PercentPopulationInfected
From Mortes
Group by Location, Population
order by PercentPopulationInfected desc
'''

In [31]:
resultado8 = pd.read_sql_query(consulta8, con)
display(resultado8)

,location,population,HighestInfectionCount,PercentPopulationInfected
0,Faeroe Islands,52888.0,34658.0,65.530933
1,Cyprus,896007.0,568383.0,63.435107
2,Gibraltar,32670.0,19999.0,61.215182
3,San Marino,33746.0,20007.0,59.287027
4,Andorra,79034.0,45793.0,57.940886
...,...,...,...,...
239,Tokelau,1849.0,NaN,NaN
240,Turkmenistan,6341855.0,NaN,NaN
241,Tuvalu,11204.0,NaN,NaN
242,United States Virgin Islands,100091.0,NaN,NaN


Não é surpreendente constatar que micropaíses como Andorra e San Marino estão entre os países com as maiores taxas de infecção. Como suas populações são pequenas, até mesmo um número baixo de infectados representa uma proporção maior em relação à população do país.

# Analisando por Regiões

#### Continentes com o maior número de mortes por população

In [32]:
consulta11 = '''
Select continent, MAX(cast(Total_deaths as int)) as TotalDeathCount
From Mortes
Where continent is not null 
Group by continent
order by TotalDeathCount desc
'''

In [33]:
resultado11 = pd.read_sql_query(consulta11, con)
display(resultado11)

,continent,TotalDeathCount
0,North America,1034152
1,South America,679996
2,Asia,526689
3,Europe,374983
4,Africa,101982
5,Oceania,12335


Não coincidentemente, América do Sul e América do Norte são os países com o maior número de mortes. è possível perceber a influencia das presenças de Brasil e estados Unidos, os dois países com o maior número de mortes no mundo.

In [34]:
# numeros globais
consulta12 = '''
Select SUM(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths, SUM(cast(new_deaths as int))/SUM(New_Cases)*100 as DeathPercentage
From Mortes
where continent is not null 
--Group By date
order by 1,2
'''

In [35]:
resultado12 = pd.read_sql_query(consulta12, con)
display(resultado12)

,total_cases,total_deaths,DeathPercentage
0,583503635.0,6376356,1.092771
